In [3]:
import faiss
from representations.contracts import contracts
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')

In [6]:
from tqdm.auto import tqdm 
for contract, paragraph_gen in tqdm(contracts.items()):
    paragraphs = list(paragraph_gen())
        

  0%|          | 0/7400 [00:00<?, ?it/s]